In [24]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
using CSV, DataFrames
using Interpolations

## Importing ShadowHand URDF

In [ ]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end


module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
shadow_cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
shadow_robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), shadow_cfg)

vm_cfg = URDFParserConfig(;suppress_warnings=true) 
vm_robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), vm_cfg) 

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

## Building the Virtual Mechanism

In [26]:
vms = VirtualMechanismSystem("myShadowVMS", shadow_robot, vm_robot)

VirtualMechanismSystem 'myShadowVMS' with robot `shadowhand_motor`, and virtual mechanism `shadowhand_motor`.

## Animating Solution

### Loading CSV

In [ ]:
df = CSV.read("shadowhand_feedback_0.001.csv", DataFrame)

t_data = df[:, 1] .- df[1, 1]  

q_data = Matrix(df[:, 2:end])  

robot_reorder_idxs = zeros(Int,24)

joint_names = ["rh_WRJ1", "rh_WRJ2", "rh_FFJ1", "rh_FFJ2", "rh_FFJ3", "rh_FFJ4", "rh_MFJ1",
                "rh_MFJ2", "rh_MFJ3", "rh_MFJ4", "rh_RFJ1", "rh_RFJ2", "rh_RFJ3", "rh_RFJ4", 
                "rh_LFJ1", "rh_LFJ2", "rh_LFJ3", "rh_LFJ4", "rh_LFJ5", "rh_THJ1", "rh_THJ2", 
                "rh_THJ3", "rh_THJ4", "rh_THJ5"]

vms_compiled = compile(vms)

for i in 1:length(joint_names)
    joint_id = get_compiled_jointID(vms_compiled.robot, joint_names[i])
    joint = vms_compiled.robot[joint_id]
    joint_idx = q_idxs(joint)
    robot_reorder_idxs[joint_idx] = i
end

vm_reorder_idxs = robot_reorder_idxs .+ 24
reorder_idxs = vcat(robot_reorder_idxs, vm_reorder_idxs)

q_data = q_data[:, reorder_idxs];

### Creation of a fake ODE solution

In [ ]:
joint_interpolations = [extrapolate(interpolate((t_data,), q_data[:, j], Gridded(Linear())), Line()) for j in 1:size(q_data, 2)]

# Custom struct to mimic an ODESolution
struct FakeODESolution
    t::Vector{Float64}
    interps::Vector{Interpolations.Extrapolation} 
end

function (sol::FakeODESolution)(t::Float64)
    return vcat([interp(t) for interp in sol.interps], zeros(length(sol.interps)))  # Return a vector of interpolated values
end

sol = FakeODESolution(t_data, joint_interpolations);


In [97]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; zoom_shift_lookat = true)
cam.lookat[] = [0.0, 0., 0.5]
cam.eyeposition[] = [0.001, -0.0, 0.5]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))

plotting_robot_kcache = map(plotting_kcache) do k
    VMRobotControl.robot_cache(k)
end
robotsketch!(ls, plotting_robot_kcache; scale = 0.02, linecolor = :red)

plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
robotsketch!(ls, plotting_vm_kcache; scale = 0.02)

savepath = joinpath(module_path, "docs/src/assets/animation_feedback_0.001.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);

┌ Info: Fastforwarding animation at 1.9530548413594564× speed, so that it plays in 15 seconds (was 29.295822620391846 seconds).
└ @ VMRobotControlMakieExt C:\Users\bilou\OneDrive\Documents\Ecole\MASTER2\TFE\Julia Simulator\VMRobotControl.jl\ext\VMRobotControlMakieExt.jl:651
